In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from scipy import *
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.cross_validation import cross_val_score, StratifiedKFold

/Users/k_martel/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
######Importation des données##########

#fight_dataframe = pd.read_csv("fights.csv",
#                   sep='\t',index_col=None,header=None,engine='python')
fight_dataframe = pd.read_csv("fights.csv",
                   sep=';',index_col=None,header=None,engine='python')

fighter_dataframe = pd.read_csv("fighter_file.csv",
                   sep='\t',index_col=None,header=None,engine='python')

In [4]:
# récuperer les infos de chaque combattant dans un dictionnaire
fighter_data={}
nb_fights=fighter_dataframe.shape[0]-1

for i in range(0,nb_fights):
    #on recupere les valeurs pour le combat le plus recent
    fighter=fighter_dataframe.values[i,0]
    nationality=fighter_dataframe.values[i,1]
    category=fighter_dataframe.values[i,2]
    poids=float(str(fighter_dataframe.values[i,3]).split(" lbs")[0])
    taille=float(str(fighter_dataframe.values[i,4]).split(" cm")[0])
    date_naissance=fighter_dataframe.values[i,5]
    
    if not pd.isnull(date_naissance) and not pd.isnull(nationality) and not pd.isnull(category) and (poids !=0) and (taille !=0):
        date_naissance=datetime.strptime(date_naissance, '%Y-%m-%d')
        fighter_data[fighter]=[nationality,category,poids,taille,date_naissance]
    

In [5]:
#faire un double dico pour chaque combattant et chaque date
fighters_fights={}
nb_fights=fight_dataframe.shape[0]-1

# a modifier avec nb_fights
for i in range(0,nb_fights):
    #on recupere les valeurs pour le combat le plus recent
    fighter=fight_dataframe.values[i,0]
    adversaire=fight_dataframe.values[i,1]
    win_or_loss=str(fight_dataframe.values[i,2])
    type_resultat=str(fight_dataframe.values[i,3])
    date_combat=fight_dataframe.values[i,6]
    #on verifie quon a le combattant et son adversaire dans le dictionnaire avant de continuer
    if (fighter in fighter_data) and (adversaire in fighter_data) and (win_or_loss=="win" or win_or_loss=="loss") and not pd.isnull(date_combat):
        
        date_combat=datetime.strptime(date_combat, '%b / %d / %Y')
        
        if win_or_loss=="win":
            resultat=1
            resultat_adv=0
        elif win_or_loss=="loss":
            resultat=0
            resultat_adv=1
                                                                    
        # si le combattant n'est pas encore présent dans le dico on réinitiale le dico des combats
        if not fighter in fighters_fights:
            fights={}
        else:# s'il est présent on récupere l'historique de ses combats et on supprime l'entrée
            fights=fighters_fights.get(fighter)
            del fighters_fights[fighter]
        
        # si l'adversaire n'est pas encore présent dans le dico on réinitiale le dico des combats
        if not adversaire in fighters_fights:
            fights_adv={}
        else:# s'il est présent on récupere l'historique de ses combats et on supprime l'entrée
            fights_adv=fighters_fights.get(adversaire)
            del fighters_fights[adversaire]
            
        fights[date_combat]=[adversaire,type_resultat,resultat]
        fights_adv[date_combat]=[fighter,type_resultat,resultat_adv]

        # mise a jour du dico des combattants avec le dico des combats
        fighters_fights[fighter]=fights
        fighters_fights[adversaire]=fights_adv




KeyboardInterrupt: 

In [10]:
# construction du vecteur pour le machine learning a-b

# creation d'un fichier 
mma_file = open('stats_fight_mma_soustraction.csv', 'w')

for fighter in fighters_fights.keys():
    for date_fight in fighters_fights.get(fighter).keys():
        donnees_combat=fighters_fights.get(fighter).get(date_fight)
        adversaire=donnees_combat[0]
        resultat=donnees_combat[2]
        
        donnees_combattant=fighter_data.get(fighter)
        donnees_combattant_adv=fighter_data.get(adversaire)
        
        if (adversaire in fighters_fights) and (date_fight in fighters_fights.get(adversaire)):
            
            # construction palmares combattant et adversaire grace aux dictionnaires de leur combats
            # recup du dictionnaire
            dict_combat_combattant =fighters_fights.get(fighter)
            dict_combat_adv=fighters_fights.get(adversaire)
            
            # on initialise les valeurs pour construire le palmares du combattant    
            victoire=0
            defaite=0
            KO=0
            soumission=0
            streak=0

            # on boucle sur ses anciens combats pour construire son palmares
            for date in (sorted(dict_combat_combattant.keys(),reverse=True)):
                if date<date_fight:
                    resultat_precedent =dict_combat_combattant.get(date)[2]
                    type_resultat_precedent=dict_combat_combattant.get(date)[1]
                    # recence le palmares a l'ufc
                    if resultat_precedent==1:
                        victoire+=1
                        # recence le type de victoire a l'ufc
                        if ("TKO" in type_resultat_precedent) or ("KO" in type_resultat_precedent):
                            KO+=1
                        elif "Submission" in type_resultat_precedent:
                            soumission+=1
                    elif resultat_precedent==0:
                        defaite+=1
                    # on construit une variable de forme (nombre de vict ou defaite consecutive)   
                    if victoire==0:
                        streak = max(victoire,defaite)
                    elif defaite==0:
                        streak = -max(victoire,defaite)
                        
            # Idem pour l'adversaire !!
            # on initialise les valeurs pour construire le palmares de l'adv    
            victoire_adv=0
            defaite_adv=0
            KO_adv=0
            soumission_adv=0
            streak_adv=0

            # on boucle sur ses anciens combats pour construire son palmares
            for date in (sorted(dict_combat_adv.keys(),reverse=True)):
                if date<date_fight:
                    resultat_precedent =dict_combat_adv.get(date)[2]
                    type_resultat_precedent=dict_combat_adv.get(date)[1]
                    # recence le palmares a l'ufc
                    if resultat_precedent==1:
                        victoire_adv+=1
                        # recence le type de victoire a l'ufc
                        if ("TKO" in type_resultat_precedent) or ("KO" in type_resultat_precedent):
                            KO_adv+=1
                        elif "Submission" in type_resultat_precedent:
                            soumission_adv+=1
                    elif resultat_precedent==0:
                        defaite_adv+=1
                    # on construit une variable de forme (nombre de vict ou defaite consecutive)   
                    if victoire_adv==0:
                        streak_adv = max(victoire_adv,defaite_adv)
                    elif defaite_adv==0:
                        streak_adv = -max(victoire_adv,defaite_adv)
            
            experience=victoire+defaite
            experience_adv=victoire_adv+defaite_adv
            diff_poids=donnees_combattant[2]-donnees_combattant_adv[2]
            diff_taille=donnees_combattant[3]-donnees_combattant_adv[3]
            diff_age=donnees_combattant[4].year-donnees_combattant_adv[4].year
            diff_experience=experience-experience_adv
            diff_victoire=victoire-victoire_adv
            diff_defaite=defaite-defaite_adv
            diff_KO=KO-KO_adv
            diff_soumission=soumission-soumission_adv
            diff_streak=streak-streak_adv
            
            # rassemblement des donnees dans un array
            data_combat=[fighter,donnees_combattant[1],donnees_combattant[0],donnees_combattant_adv[0],
                         donnees_combattant[2],donnees_combattant[3],donnees_combattant[4].year,
                         experience,victoire,defaite,KO,soumission,streak,
                         donnees_combattant_adv[2],donnees_combattant_adv[3],donnees_combattant_adv[4].year,
                         experience_adv,victoire_adv,defaite_adv,KO_adv,soumission_adv,streak_adv,
                        diff_poids,diff_taille,diff_age,
                        diff_victoire,diff_defaite,diff_KO,diff_soumission,diff_streak,
                        resultat,adversaire]
            mma_file.write("\t".join([str(d) for d in data_combat]))
            mma_file.write("\n")
    

In [13]:
# construction du vecteur pour le machine learning a/b

# creation d'un fichier 
mma_file = open('stats_fight_mma_division.csv', 'w')

for fighter in fighters_fights.keys():
    for date_fight in fighters_fights.get(fighter).keys():
        donnees_combat=fighters_fights.get(fighter).get(date_fight)
        adversaire=donnees_combat[0]
        resultat=donnees_combat[2]
        
        donnees_combattant=fighter_data.get(fighter)
        donnees_combattant_adv=fighter_data.get(adversaire)
        
        if (adversaire in fighters_fights) and (date_fight in fighters_fights.get(adversaire)):
            
            # construction palmares combattant et adversaire grace aux dictionnaires de leur combats
            # recup du dictionnaire
            dict_combat_combattant =fighters_fights.get(fighter)
            dict_combat_adv=fighters_fights.get(adversaire)
            
            # on initialise les valeurs pour construire le palmares du combattant    
            victoire=0
            defaite=0
            KO=0
            soumission=0
            streak=0

            # on boucle sur ses anciens combats pour construire son palmares
            for date in (sorted(dict_combat_combattant.keys(),reverse=True)):
                if date<date_fight:
                    resultat_precedent =dict_combat_combattant.get(date)[2]
                    type_resultat_precedent=dict_combat_combattant.get(date)[1]
                    # recence le palmares a l'ufc
                    if resultat_precedent==1:
                        victoire+=1
                        # recence le type de victoire a l'ufc
                        if ("TKO" in type_resultat_precedent) or ("KO" in type_resultat_precedent):
                            KO+=1
                        elif "Submission" in type_resultat_precedent:
                            soumission+=1
                    elif resultat_precedent==0:
                        defaite+=1
                    # on construit une variable de forme (nombre de vict ou defaite consecutive)   
                    if victoire==0:
                        streak = max(victoire,defaite)
                    elif defaite==0:
                        streak = -max(victoire,defaite)
                        
            # Idem pour l'adversaire !!
            # on initialise les valeurs pour construire le palmares de l'adv    
            victoire_adv=0
            defaite_adv=0
            KO_adv=0
            soumission_adv=0
            streak_adv=0

            # on boucle sur ses anciens combats pour construire son palmares
            for date in (sorted(dict_combat_adv.keys(),reverse=True)):
                if date<date_fight:
                    resultat_precedent =dict_combat_adv.get(date)[2]
                    type_resultat_precedent=dict_combat_adv.get(date)[1]
                    # recence le palmares a l'ufc
                    if resultat_precedent==1:
                        victoire_adv+=1
                        # recence le type de victoire a l'ufc
                        if ("TKO" in type_resultat_precedent) or ("KO" in type_resultat_precedent):
                            KO_adv+=1
                        elif "Submission" in type_resultat_precedent:
                            soumission_adv+=1
                    elif resultat_precedent==0:
                        defaite_adv+=1
                    # on construit une variable de forme (nombre de vict ou defaite consecutive)   
                    if victoire_adv==0:
                        streak_adv = max(victoire_adv,defaite_adv)
                    elif defaite_adv==0:
                        streak_adv = -max(victoire_adv,defaite_adv)
                        
            experience=victoire+defaite
            experience_adv=victoire_adv+defaite_adv
            diff_poids=donnees_combattant[2]/donnees_combattant_adv[2]
            diff_taille=donnees_combattant[3]/donnees_combattant_adv[3]
            diff_age=donnees_combattant[4].year/donnees_combattant_adv[4].year
            
            # retraitement des données pour eviter les erreurs / 0
            if victoire_adv!=0:
                diff_victoire=victoire/victoire_adv
            else:
                diff_victoire=0
                
            if defaite_adv!=0:
                diff_defaite=defaite/defaite_adv
            else:
                diff_defaite=0
                
            if KO_adv!=0:
                diff_KO=KO/KO_adv
            else:
                diff_KO=0
                
            if soumission_adv!=0:
                diff_soumission=soumission/soumission_adv
            else:
                diff_soumission=0
                
            if streak_adv!=0:
                diff_streak=streak/streak_adv
            else:
                diff_streak=0
                
            # rassemblement des donnees dans un array
            data_combat=[fighter,donnees_combattant[1],donnees_combattant[0],donnees_combattant_adv[0],
                         donnees_combattant[2],donnees_combattant[3],donnees_combattant[4].year,
                         experience,victoire,defaite,KO,soumission,streak,
                         donnees_combattant_adv[2],donnees_combattant_adv[3],donnees_combattant_adv[4].year,
                         experence_adv,victoire_adv,defaite_adv,KO_adv,soumission_adv,streak_adv,
                         diff_poids,diff_taille,diff_age,
                         diff_victoire,diff_defaite,diff_KO,diff_soumission,diff_streak,
                         resultat,adversaire]
            mma_file.write("\t".join([str(d) for d in data_combat]))
            mma_file.write("\n")
    

In [14]:
# construction du vecteur pour le machine learning (a-b)/(a+b)

# creation d'un fichier 
mma_file = open('stats_fight_mma_mapping.csv', 'w')

for fighter in fighters_fights.keys():
    for date_fight in fighters_fights.get(fighter).keys():
        donnees_combat=fighters_fights.get(fighter).get(date_fight)
        adversaire=donnees_combat[0]
        resultat=donnees_combat[2]
        
        donnees_combattant=fighter_data.get(fighter)
        donnees_combattant_adv=fighter_data.get(adversaire)
        
        if (adversaire in fighters_fights) and (date_fight in fighters_fights.get(adversaire)):
            
            # construction palmares combattant et adversaire grace aux dictionnaires de leur combats
            # recup du dictionnaire
            dict_combat_combattant =fighters_fights.get(fighter)
            dict_combat_adv=fighters_fights.get(adversaire)
            
            # on initialise les valeurs pour construire le palmares du combattant    
            victoire=0
            defaite=0
            KO=0
            soumission=0
            streak=0

            # on boucle sur ses anciens combats pour construire son palmares
            for date in (sorted(dict_combat_combattant.keys(),reverse=True)):
                if date<date_fight:
                    resultat_precedent =dict_combat_combattant.get(date)[2]
                    type_resultat_precedent=dict_combat_combattant.get(date)[1]
                    # recence le palmares a l'ufc
                    if resultat_precedent==1:
                        victoire+=1
                        # recence le type de victoire a l'ufc
                        if ("TKO" in type_resultat_precedent) or ("KO" in type_resultat_precedent):
                            KO+=1
                        elif "Submission" in type_resultat_precedent:
                            soumission+=1
                    elif resultat_precedent==0:
                        defaite+=1
                    # on construit une variable de forme (nombre de vict ou defaite consecutive)   
                    if victoire==0:
                        streak = max(victoire,defaite)
                    elif defaite==0:
                        streak = -max(victoire,defaite)
                        
            # Idem pour l'adversaire !!
            # on initialise les valeurs pour construire le palmares de l'adv    
            victoire_adv=0
            defaite_adv=0
            KO_adv=0
            soumission_adv=0
            streak_adv=0

            # on boucle sur ses anciens combats pour construire son palmares
            for date in (sorted(dict_combat_adv.keys(),reverse=True)):
                if date<date_fight:
                    resultat_precedent =dict_combat_adv.get(date)[2]
                    type_resultat_precedent=dict_combat_adv.get(date)[1]
                    # recence le palmares a l'ufc
                    if resultat_precedent==1:
                        victoire_adv+=1
                        # recence le type de victoire a l'ufc
                        if ("TKO" in type_resultat_precedent) or ("KO" in type_resultat_precedent):
                            KO_adv+=1
                        elif "Submission" in type_resultat_precedent:
                            soumission_adv+=1
                    elif resultat_precedent==0:
                        defaite_adv+=1
                    # on construit une variable de forme (nombre de vict ou defaite consecutive)   
                    if victoire_adv==0:
                        streak_adv = max(victoire_adv,defaite_adv)
                    elif defaite_adv==0:
                        streak_adv = -max(victoire_adv,defaite_adv)
                        
            experience=victoire+defaite
            experience_adv=victoire_adv+defaite_adv
            
            diff_poids=(donnees_combattant[2]-donnees_combattant_adv[2])/(donnees_combattant[2]+donnees_combattant_adv[2])
            diff_taille=(donnees_combattant[3]-donnees_combattant_adv[3])/(donnees_combattant[3]+donnees_combattant_adv[3])
            diff_age=(donnees_combattant[4].year-donnees_combattant_adv[4].year)/(donnees_combattant[4].year+donnees_combattant_adv[4].year)
            
            # retraitement des données pour eviter les erreurs / 0
            if (victoire+victoire_adv)!=0:
                diff_victoire=(victoire-victoire_adv)/(victoire+victoire_adv)
            else:
                diff_victoire=0
                
            if (defaite+defaite_adv)!=0:
                diff_defaite=(defaite-defaite_adv)/(defaite+defaite_adv)
            else:
                diff_defaite=0
                
            if (KO+KO_adv)!=0:
                diff_KO=(KO-KO_adv)/(KO+KO_adv)  
            else:
                diff_KO=0
                
            if (soumission+soumission_adv)!=0:
                diff_soumission=(soumission-soumission_adv)/(soumission+soumission_adv)
            else:
                diff_soumission=0
                
            if (streak+streak_adv)!=0:
                diff_streak=(streak-streak_adv)/(streak+streak_adv)
            else:
                diff_streak=0
                
           
            # rassemblement des donnees dans un array
            data_combat=[fighter,donnees_combattant[1],donnees_combattant[0],donnees_combattant_adv[0],
                         donnees_combattant[2],donnees_combattant[3],donnees_combattant[4].year,
                         experience,victoire,defaite,KO,soumission,streak,
                         donnees_combattant_adv[2],donnees_combattant_adv[3],donnees_combattant_adv[4].year,
                         experence_adv,victoire_adv,defaite_adv,KO_adv,soumission_adv,streak_adv,
                        diff_poids,diff_taille,diff_age,
                        diff_victoire,diff_defaite,diff_KO,diff_soumission,diff_streak,
                        resultat,adversaire]
            mma_file.write("\t".join([str(d) for d in data_combat]))
            mma_file.write("\n")
    